# Gemini 텍스트 생성

* 모델 종류: https://ai.google.dev/gemini-api/docs/models/gemini?hl=ko&_gl=1

## 텍스트 전용 입력에서 텍스트 생성

In [ ]:
from google import genai

client = genai.Client(api_key="GEMINI_API_KEY")

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["How does AI work?"])
print(response.text)

## 텍스트 및 이미지 입력에서 텍스트 생성

* Gemini API는 텍스트와 미디어 파일을 결합한 멀티모달 입력을 지원합니다. 다음 예는 텍스트 및 이미지 입력에서 텍스트를 생성하는 방법을 보여줍니다.

In [ ]:
from PIL import Image
from google import genai

client = genai.Client(api_key="GEMINI_API_KEY")

image = Image.open("/path/to/organ.png")
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[image, "Tell me about this instrument"])
print(response.text)

## 텍스트 스트림 생성

* 기본적으로 모델은 전체 텍스트 생성 프로세스를 완료한 후에 응답을 반환합니다. 전체 결과를 기다리지 않고 대신 스트리밍을 사용하여 부분 결과를 처리하면 더 빠른 상호작용을 실행할 수 있습니다.
* 다음 예에서는 streamGenerateContent 메서드를 사용하여 스트리밍을 구현하여 텍스트 전용 입력 프롬프트에서 텍스트를 생성하는 방법을 보여줍니다.

In [ ]:
from google import genai

client = genai.Client(api_key="GEMINI_API_KEY")

response = client.models.generate_content_stream(
    model="gemini-2.0-flash",
    contents=["Explain how AI works"])
for chunk in response:
    print(chunk.text, end="")

## 채팅 대화 만들기

* Gemini SDK를 사용하면 여러 번의 질문과 응답을 수집할 수 있으므로 사용자가 점진적으로 답변을 찾거나 여러 부분으로 구성된 문제와 관련하여 도움을 받을 수 있습니다. 이 SDK 기능은 대화 기록을 추적하는 인터페이스를 제공하지만, 백그라운드에서는 동일한 generateContent 메서드를 사용하여 응답을 만듭니다.

* 다음 코드 예는 기본 채팅 구현을 보여줍니다.

In [ ]:
from google import genai

client = genai.Client(api_key="GEMINI_API_KEY")

chat = client.chats.create(model="gemini-2.0-flash")
response = chat.send_message("I have 2 dogs in my house.")
print(response.text)
response = chat.send_message("How many paws are in my house?")
print(response.text)
for message in chat._curated_history:
    print(f'role - ', message.role, end=": ")
    print(message.parts[0].text)

* 다음 예와 같이 채팅과 함께 스트리밍을 사용할 수도 있습니다.

In [ ]:
from google import genai

client = genai.Client(api_key="GEMINI_API_KEY")

chat = client.chats.create(model="gemini-2.0-flash")
response = chat.send_message_stream("I have 2 dogs in my house.")
for chunk in response:
    print(chunk.text, end="")
response = chat.send_message_stream("How many paws are in my house?")
for chunk in response:
    print(chunk.text, end="")
for message in chat._curated_history:
    print(f'role - ', message.role, end=": ")
    print(message.parts[0].text)

## 텍스트 생성 구성

* 모델에 전송하는 모든 프롬프트에는 모델이 응답을 생성하는 방식을 제어하는 매개변수가 포함됩니다. GenerationConfig를 사용하여 이러한 매개변수를 구성할 수 있습니다. 매개변수를 구성하지 않으면 모델은 기본 옵션을 사용하며 이는 모델에 따라 다를 수 있습니다.

* 다음 예는 사용 가능한 여러 옵션을 구성하는 방법을 보여줍니다.

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key="GEMINI_API_KEY")

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["Explain how AI works"],
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0.1
    )
)
print(response.text)

## 시스템 안내 추가

* 시스템 안내를 사용하면 특정 요구사항 및 사용 사례에 따라 모델의 동작을 조정할 수 있습니다.

* 모델에 시스템 안내를 제공하면 작업을 이해하고, 보다 맞춤설정된 대답을 생성하고, 모델과의 전체 사용자 상호작용에 대한 특정 가이드라인을 준수하기 위한 추가 컨텍스트를 모델에 제공할 수 있습니다. 최종 사용자가 제공하는 프롬프트와 별도로 시스템 안내를 설정하여 제품 수준 동작을 지정할 수도 있습니다.

* 모델을 초기화할 때 시스템 안내를 설정할 수 있습니다.

In [ ]:
sys_instruct="You are a cat. Your name is Neko."
client = genai.Client(api_key="GEMINI_API_KEY")

response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=sys_instruct),
    contents=["your prompt here"]
)